In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY") 
GROQ_API_KEY= os.environ.get("GROQ_API_KEY") 

### Selección de imagen

In [ ]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
# Path to your image
image_path = "doc.png"

# Getting the base64 string
base64_image = encode_image(image_path)

### Esquema del objeto

In [ ]:
from pydantic import BaseModel, Field

class InfoDOM(BaseModel):
    departamento: str = Field(description="Nombre del departamento")
    municipio: str = Field(description="Nombre del municipio")
    puesto: str = Field(description="Número del puesto")
    zona: str = Field(description="Número de la zona")
    mesa: str = Field(description="Número de la mesa")
    votosGustavo: int = Field(description="Número de votos para Gustavo Petro")
    votosIvan: int = Field(description="Número de votos para Ivan Duque")
    votosBlanco: int = Field(description="Número de votos en Blanco")
    votosNulos: int = Field(description="Número de votos en Nulos")
    votosNoMarcados: int = Field(description="Número de votos en No Marcados")
    votosTotal: int = Field(description="Número total de motos de la mesa")
    votosSufragantes: int = Field(description="Número total de votos sufragantes")
    votosUrna: int = Field(description="Número total de votos en la urna")
    votosIncinerados: int = Field(description="Número total de votos incinerados")


## Extracción de imágenes

Mensajes

### Modelo de OpenAI

In [49]:
messages = [
    {'role': 'system', "content" : f"""
     You will extract text from the next image, where there are some votes for different political parties. 
     The text is in Spanish. The output should be in JSON format following the next structure:
     {InfoDOM.schema()}"""},
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Extract the information of this document"},
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
]

In [58]:
from openai import OpenAI

def getClientOpenAI():
    local = False
    if local:
        client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=GROQ_API_KEY)
        model = "llama-3.2-11b-vision-preview"
    else:
        client = OpenAI()
        model = "gpt-4o"
    return client, model

client, model = getClientOpenAI()

In [60]:
chat_completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    temperature=0,
    response_format=InfoDOM
    # response_format={"type": "json_object"}#,"json_schema": {"name": "info_votos","schema":InfoDOM.schema()}}
)

In [61]:
import json

json.loads(chat_completion.choices[0].message.content)

{'departamento': 'Antioquia',
 'municipio': 'Medellin',
 'puesto': '01',
 'zona': '01',
 'mesa': '001',
 'votosGustavo': 24,
 'votosIvan': 164,
 'votosBlanco': 1,
 'votosNulos': 1,
 'votosNoMarcados': 1,
 'votosTotal': 191,
 'votosSufragantes': 191,
 'votosUrna': 191,
 'votosIncinerados': 0}

### Usando GROQ y modelos locales

In [ ]:
messages_groq = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": f"""
     You will extract text from the next image, where there are some votes for different political parties. 
     The text is in Spanish. The output should be in JSON format following the next structure:
     {InfoDOM.schema()}"""},
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
]

In [45]:
from groq import Groq

client_groq = Groq()

chat_completion = client_groq.chat.completions.create(
    messages=messages_groq,
    model=model,
    temperature=0,
    response_format={"type": "json_object"}
)

In [43]:
print(chat_completion.choices[0].message.content)

{
   "properties": {
      "departamento": {
         "description": "Nombre del departamento",
         "title": "Departamento",
         "type": "string"
      },
      "municipio": {
         "description": "Nombre del municipio",
         "title": "Municipio",
         "type": "string"
      },
      "puesto": {
         "description": "Número del puesto",
         "title": "Puesto",
         "type": "string"
      },
      "zona": {
         "description": "Número de la zona",
         "title": "Zona",
         "type": "string"
      },
      "mesa": {
         "description": "Número de la mesa",
         "title": "Mesa",
         "type": "string"
      },
      "votosGustavo": {
         "description": "Número de votos para Gustavo Petro",
         "title": "Votosgustavo",
         "type": "integer"
      },
      "votosIvan": {
         "description": "Número de votos para Ivan Duque",
         "title": "Votosivan",
         "type": "integer"
      },
      "votosBlanco": {
    